In [1]:
# importing required modules
import PyPDF2
  
# creating a pdf file object
pdfFileObj = open('ComputingScienceSummaries.pdf', 'rb')
  
# creating a pdf reader object
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
  
text = ''

for page in range(pdfReader.numPages):
    pageObj = pdfReader.getPage(page)
    text += pageObj.extractText()
#pageObj = pdfReader.getPage(0)
  
# closing the pdf file object
pdfFileObj.close()

In [2]:
courses = {}
for line in text.splitlines():
    print(line)

 
 
CT S ,  B IT :  CS E  S U M M A RY  /  
1
 
© Alberta Education, Alberta,Canada
 
 
Revi sed 
2010
 
CO MP UT I NG  S CI E NCE  
CO URS E  S UMM AR I E S
 
 
I
NT RODUCT ORY
 
 
C SE 101 0:
 
C OM PU T ER  SC I EN C E 1
 
 
Stud e nts e xp lo r e har d war e,  so ft war e and  p r o cesses.
 
T his i nc lud e s a n i ntr o d uc tio n to  the  a l go r ith m a s a  p r o b le m
-
s o l vi n g 
tool, to programmi ng languages in general
 
a nd  to  the  r o le  
of programmi ng a s a tool for i mple ment ing al gorithms.
 
 
P re re q uis ite :
 
None
 
 
C SE 111 0:
 
ST RUCT URED PRO G R AM M ING  1
 
 
Stud e nts a r e  intr o d uc e d  to  a  g e ne r a l p r o gr a mmin g e nvir o n m e nt in wh ic h t he y wr ite  si mp le  str uc t ur e d  a lgo r ith ms a nd  
p r o gr a ms  t ha t  i np ut ,  p r o c e s s  a nd
 
o utp ut d ata,  u se so me o f th e mo r e b asic o p er ato r s and  d ata t yp es,  and  fo llo w a 
se q ue n tia l f lo w o f c o ntr o l.
 
 
P re re q uis i